In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.8 MB/s eta 0:00:00


In [2]:
import pathlib
import google.generativeai as genai

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
model = genai.GenerativeModel('gemini-1.0-pro-001')

In [6]:
# response = model.generate_content(
#     '''
#     ''')

In [7]:
# text = (response.text)
# print(text)

In [38]:
import os
import pandas as pd
from tqdm import tqdm
import time

def getGemini10Pro001Response(prompt):
    # sleep to avoid api limits, should take ~15 minutes in total
    # time.sleep(1)
    response = model.generate_content(prompt).text
    return response

def getLLMResponse(prompt):
    return getGemini10Pro001Response(prompt)

def getGeminiChatResponse(model,question,subquestions): #model type / large question in str/ subquestions in array

    header_instruction = ''' ''' ##### the one for starting the chat
    final_instruction = ''' list all my advantage on basketball''' ### what's the best answer etc...
    chat = model.start_chat(history=[])
    response = chat.send_message(header_instruction + question)
    for subs in subquestions:
        response = chat.send_message(subs)
    response  = chat.send_message(final_instruction)
    return response.text

In [39]:
getGeminiChatResponse(model,"i will gave you some information first, and then ask you a quesrion",["i'm 7,0","i'm 280lb","i can run 100m in 10s","i can shoot 3 point 100 in a row"])

'**Physical Advantages:**\n\n* **Height:** Being 7 feet tall gives you a significant advantage in terms of reach, rebounding, and shot blocking.\n* **Weight:** Weighing 280 pounds provides you with strength and power, which can be beneficial for post play and driving to the basket.\n* **Speed:** Your ability to run 100 meters in 10 seconds indicates that you have good speed and agility, which can help you get past defenders and create scoring opportunities.\n\n**Skills Advantages:**\n\n* **Three-point shooting:** Your ability to make 100 three-point shots in a row demonstrates your exceptional shooting accuracy and range. This skill allows you to spread the floor and create scoring opportunities for yourself and your teammates.\n* **Rebounding:** Your height and strength give you a natural advantage in rebounding. You can position yourself well to grab rebounds and create second-chance scoring opportunities for your team.\n* **Shot blocking:** Your height and reach make you a formidabl

In [24]:
# def getBaseAnswers(dataDf):
#     BASE_ANS_DIR = './Gemini_results.json'

#     saveDf = None#pd.DataFrame(columns=['question', 'reasoning', 'subproblems', 'label'])
#     #saveDf = #saveDf.astype(str)

#     for index, row in tqdm(dataDf.iterrows(), total=len(dataDf)):
#         prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
#         response = getLLMResponse(prompt)
#         # if saveDf is None:
#         #     saveDf = pd.DataFrame(columns=['question', 'reasoning', 'subproblems', 'label'])
#         #     saveDf = saveDf.astype(str)
#         # else:
#         #     saveDf = pd.read_json(BASE_ANS_DIR, dtype=str)

#         # new_record = {
#         #     'question': row['example'],
#         #     'gold-reasoning': row['proof'],
#         #     'subproblems': '',
#         #     'chat-gpt-3.5-turbo-ans': response,
#         #     'label': row['label'],
#         # }
#         # temp_df = pd.DataFrame([new_record])
#         # saveDf = pd.concat([saveDf, temp_df], ignore_index=True)
#         # saveDf.to_json(BASE_ANS_DIR, orient='records')
import csv
import json
def getBaseAnswers(dataDf):
    output_csv_path = '../data/Gemini_results.csv'

    # Open the CSV file for writing
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        #the header row
        writer.writerow(['Example', 'Response'])
        count = 0
        for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
            # Construct the prompt 
            prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
            # Get the response 
            response = getLLMResponse(prompt)
            # Write to the CSV
            writer.writerow([row['example'], response])
            count += 1
            print("The response for " + str(count) + "is: ",response)

In [ ]:
def main():
    dataPath = "../data/test.json"

    df = pd.read_json(dataPath, dtype=str)
    getBaseAnswers(df)

if __name__ == "__main__":
    main()


In [ ]:
# chat = model.start_chat(history=[])

In [29]:
# response = chat.send_message("do u know donald trump?")
# response.text

'Yes, I know Donald Trump. He is the 45th and current president of the United States. He was born on June 14, 1946, in Queens, New York City. He is a businessman and television personality. He is the founder and former chairman of The Trump Organization, a real estate development company. He is also the founder of Trump Entertainment Resorts, which owned several casinos and hotels. He is the author of several books, including The Art of the Deal and The Art of the Comeback. He was the host of the reality television show The Apprentice and its spin-off The Celebrity Apprentice. He was elected president in 2016, defeating Hillary Clinton. He is married to Melania Trump, and they have one child together.'